In [1]:
! which python

/Users/no_lineal/opt/anaconda3/envs/research36/bin/python


In [2]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, recall_score

from sklearn.feature_selection import SelectKBest, f_classif

import os

In [3]:
"""

    where am i?

"""

PATH = os.getcwd() + '/'
data_path = PATH + 'data/'

print(f'PATH: {PATH}')
print(f'data path: {data_path}')

PATH: /Users/no_lineal/Documents/GitHub/eeg_processing/
data path: /Users/no_lineal/Documents/GitHub/eeg_processing/data/


In [4]:
"""

    load data

"""

filename = 'lfp_features.csv'
data = pd.read_csv( data_path + filename )

print(f'\ndataset: {filename}\n')
print(data.info())
data.head()


dataset: lfp_features.csv

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1241 entries, 0 to 1240
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   delta_power   1241 non-null   float64
 1   theta_power   1241 non-null   float64
 2   alpha_power   1241 non-null   float64
 3   beta_power    1241 non-null   float64
 4   gamma_power   1241 non-null   float64
 5   std           1241 non-null   float64
 6   mean          1241 non-null   float64
 7   skewness      1241 non-null   float64
 8   h_mobility    1241 non-null   float64
 9   h_complexity  1241 non-null   float64
 10  subject       1241 non-null   object 
 11  label         1241 non-null   int64  
dtypes: float64(10), int64(1), object(1)
memory usage: 116.5+ KB
None


,delta_power,theta_power,alpha_power,beta_power,gamma_power,std,mean,skewness,h_mobility,h_complexity,subject,label
0,0.0,0.0,4758.662269,2640.061771,364.112271,453.066877,-8.535503,-0.093628,0.028487,47.397276,STN02,0
1,0.0,0.0,442.979556,341.863066,155.905377,184.311566,37.425427,0.519701,0.065246,23.446911,STN02,0
2,0.0,0.0,712.150198,232.794823,246.771257,128.088896,24.177393,-0.158139,0.107012,13.711945,STN02,0
3,0.0,0.0,1074.317644,480.023532,190.766989,111.534242,22.095954,-0.060430,0.126149,12.349120,STN02,0
4,0.0,0.0,452.945133,282.104553,124.667279,84.057426,-4.245994,0.348994,0.168331,8.902656,STN02,0


In [5]:
"""

    train vector

"""

subject = ''
#subject = np.random.choice( data['subject'].unique(), 1 )
print(data['subject'].unique())
#subject = [ sorted( data['subject'].unique() )[6] ]

if subject:
    
    print(f'subject: {subject}')
    data = data[ data['subject'].isin( subject ) ]
    
features = [ x for x in list( data ) if x not in ['subject', 'label', 'delta_power', 'theta_power'] ]

print(f'features: \n{features}')

X = data[ features ]
y = data[ 'label' ].values

print(f'X: {X.shape}')
print(f'y: {y.shape}')

['STN02' 'STN07a' 'STN08' 'STN10a' 'STN10b' 'STN11' 'STN13']
features: 
['alpha_power', 'beta_power', 'gamma_power', 'std', 'mean', 'skewness', 'h_mobility', 'h_complexity']
X: (1241, 8)
y: (1241,)


In [6]:
"""

    validation

"""

# define the feature selection
k_best = 5
fs = SelectKBest(score_func=f_classif, k=k_best)

# apply feature selection to the data
X_fs = fs.fit_transform(X, y)

# define the models
rf_model = RandomForestClassifier()
lr_model = LogisticRegression()

# define the hyperparameters to tune
rf_params = {'n_estimators': [100, 200, 500],
             'max_depth': [5, 10, 15]}
lr_params = {'C': [0.1, 1, 10],
             'penalty': ['l1', 'l2']}

# define stratified k-fold cross validation
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# compare models using ROC and recall metrics
scoring = {'roc': 'roc_auc', 'recall': 'recall'}

# tune hyperparameters using grid search
rf_grid = GridSearchCV(rf_model, rf_params, cv=skf, scoring=scoring['roc'], n_jobs=-1)
lr_grid = GridSearchCV(lr_model, lr_params, cv=skf, scoring=scoring['roc'], n_jobs=-1)

# fit the models with the best hyperparameters and selected features
rf_grid.fit(X_fs, y)
lr_grid.fit(X_fs, y)

rf_model = rf_grid.best_estimator_
lr_model = lr_grid.best_estimator_

# evaluate the models using cross-validation
rf_roc = cross_val_score(rf_model, X_fs, y, cv=skf, scoring=scoring['roc'])
rf_recall = cross_val_score(rf_model, X_fs, y, cv=skf, scoring=scoring['recall'])

lr_roc = cross_val_score(lr_model, X_fs, y, cv=skf, scoring=scoring['roc'])
lr_recall = cross_val_score(lr_model, X_fs, y, cv=skf, scoring=scoring['recall'])

# print the results
print("Random Forest ROC: {:.3f} (+/- {:.3f})".format(np.mean(rf_roc), np.std(rf_roc)))
print("Random Forest Recall: {:.3f} (+/- {:.3f})".format(np.mean(rf_recall), np.std(rf_recall)))
print("Random Forest Best Params: ", rf_grid.best_params_)

print("Logistic Regression ROC: {:.3f} (+/- {:.3f})".format(np.mean(lr_roc), np.std(lr_roc)))
print("Logistic Regression Recall: {:.3f} (+/- {:.3f})".format(np.mean(lr_recall), np.std(lr_recall)))
print("Logistic Regression Best Params: ", lr_grid.best_params_)

# get the feature scores and p-values
scores = fs.scores_
pvalues = fs.pvalues_

# create a dataframe with the scores and p-values
scores_df = pd.DataFrame({'Feature': X.columns, 'Score': scores, 'P-Value': pvalues})

# sort the features by score and print the top k features
scores_df = scores_df.sort_values(by='Score', ascending=False)
print('\nTop {} features:'.format(k_best))
print(scores_df.head(k_best))

/Users/no_lineal/opt/anaconda3/envs/research36/lib/python3.6/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan 0.49552104        nan 0.49729187        nan 0.49736999]
  category=UserWarning


Random Forest ROC: 0.491 (+/- 0.046)
Random Forest Recall: 0.425 (+/- 0.055)
Random Forest Best Params:  {'max_depth': 15, 'n_estimators': 200}
Logistic Regression ROC: 0.497 (+/- 0.042)
Logistic Regression Recall: 0.178 (+/- 0.051)
Logistic Regression Best Params:  {'C': 10, 'penalty': 'l2'}

Top 5 features:
        Feature     Score   P-Value
7  h_complexity  1.369268  0.242163
0   alpha_power  1.364026  0.243066
1    beta_power  1.018302  0.313119
3           std  0.701772  0.402350
2   gamma_power  0.679298  0.409987
